In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("/home/as/datasets/mnist", one_hot=True)

Extracting /home/as/datasets/mnist/train-images-idx3-ubyte.gz
Extracting /home/as/datasets/mnist/train-labels-idx1-ubyte.gz
Extracting /home/as/datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/as/datasets/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
def weight_var(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_var(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [4]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [5]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [6]:
with tf.name_scope("Conv1"):
    W_conv1 = weight_var([5, 5, 1, 32])
    b_conv1 = bias_var([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    
    tf.summary.histogram("weights", W_conv1)
    tf.summary.histogram("biases", b_conv1)
    tf.summary.histogram("activation", h_conv1)
    

In [7]:
with tf.name_scope("Conv2"):
    W_conv2 = weight_var([5,5,32,64])
    b_conv2 = bias_var([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    
    tf.summary.histogram("weights", W_conv2)
    tf.summary.histogram("biases", b_conv2)
    tf.summary.histogram("activation", h_conv2)

In [8]:
with tf.name_scope("Fc1"):
    W_fc1 = weight_var([7 * 7 * 64, 1024])
    b_fc1 = bias_var([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [9]:
with tf.name_scope("Fc2"):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    W_fc2 = weight_var([1024, 10])
    b_fc2 = bias_var([10])
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [10]:
with tf.name_scope("xentropy"):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv))
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
with tf.name_scope("correct_pred"):
    correct_pred = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
with tf.name_scope("accuracy"):
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [11]:
summary_acc = tf.summary.scalar("accuracy", accuracy)
summary_loss = tf.summary.scalar("xloss", cross_entropy)
summary_imgs = tf.summary.image("imgs", x_image, 3)

In [12]:
summary_merge = tf.summary.merge_all()

In [13]:
!rm -rf tensorboard/*

In [14]:
with tf.Session() as sess:
    
    writer = tf.summary.FileWriter('tensorboard/0', sess.graph)
    
    sess.run(tf.global_variables_initializer())
    for i in range(600):
        batch = mnist.train.next_batch(1000)
        train_step.run(feed_dict={x: batch[0], y: batch[1], keep_prob:0.5})
        
        if i % 10 == 0:
            test_accuracy = accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob:1})
            print(test_accuracy)
            summ_str = sess.run(summary_merge, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob:1})
            writer.add_summary(summ_str, i)
            
writer.close()

0.1339
0.4604
0.6435
0.7358
0.7942
0.8436
0.8638
0.8833
0.8953
0.9069
0.9144
0.9204
0.9258
0.931
0.9339
0.9358
0.9404
0.9429
0.9454
0.9474
0.9471
0.9502
0.9507
0.9531
0.9532
0.9529
0.9568
0.9577
0.96
0.96
0.9609
0.9618
0.9619
0.9635
0.9628
0.9643
0.9648
0.9647
0.9671
0.967
0.9677
0.9676
0.9683
0.9688
0.9691
0.9704
0.9703
0.969
0.9716
0.9691
0.9727
0.9706
0.9729
0.9739
0.9724
0.973
0.9746
0.974
0.9728
0.9739


In [ ]:
# tensorboard --logdir tensorboard/0/